In [1]:
import nltk
import re
import numpy as np
import random
from nltk import ngrams
from collections import Counter
import math

In [2]:
def preprocessing(lines):
    lines = lines.replace('\r\n',' ')
    lines = lines.replace(':'," ")
    lines = lines.replace(';'," ")
    lines = lines.replace("'","")
    lines = lines.replace('"',"")
    lines = lines.replace('-'," ")
    lines = lines.replace(')'," ")
    lines = lines.replace('('," ")
    lines = lines.replace('?',".")
    lines = lines.replace('!',".")
    lines = lines.replace('_'," ")
    return lines


In [3]:
file_name = 'alice.txt'
lines = open(file_name).read()
lines = preprocessing(lines)

In [4]:
sentences = nltk.sent_tokenize(lines)

In [5]:
sentences = [i.replace('.',"") for i in sentences if len(i.split())>=2]
words=[]
words = [words + (nltk.word_tokenize(i)) for i in sentences]

In [6]:
tagged_sentences = ["<s> "+i +" </s>" for i in sentences]

In [7]:
print (tagged_sentences[18])

<s> I wonder how many miles Ive fallen by this time </s>


In [8]:
l = len(sentences)
train_data = tagged_sentences[:int(l*.8)]
test_data = tagged_sentences[int(l*.8):]

# Unigram

In [9]:
unigram_lst=[]
for i in train_data:
    n = 1
    grams= ngrams(i.split(), n)
    for i in grams:
        unigram_lst.append(i)

In [10]:
unigram_ctr = Counter(unigram_lst)
s = sum(unigram_ctr.values())

In [11]:
unigram_mle={}
for i in unigram_ctr:
    unigram_mle[i] = unigram_ctr[i]*1.0/s

In [12]:
def unigram_sentence_probability(sentence,unigram_mle):
    
    try:

        p=1
        sentence = preprocessing(sentence)
        sentence = "<s> "+sentence+" </s>"
        keys =unigram_mle.keys()
        for i in sentence.split():
            if (i,) in keys:
                p=p*unigram_mle[(i,)]
            else:
                return 0
        return math.log(p)
    
    except:
        return 0


def unigram_sentence_generator(unigram_mle):
    keys = np.array(unigram_mle.keys())
    vals = np.array(unigram_mle.values())
    sampled_word = keys[np.random.multinomial(1,vals)==1][0][0]
    s=''+sampled_word
    cnt=0
    while True:
        sampled_word = keys[np.random.multinomial(1,vals)==1][0][0]
        if sampled_word=="</s>":
            break
        s=s+" "+sampled_word
    return s

In [13]:
unigram_sentence_probability('I am rabbit',unigram_mle)

-26.04552361263259

# Unigram MLE

In [1]:
#unigram_mle

# Unigram Generator

In [15]:
for i in range(5):
    sent = unigram_sentence_generator(unigram_mle)
    print ("\n"+sent)


</s> the smiling <s> would it not not but and with now quite this last it Such each hurried

the the <s> over a tone her sat riddle Thank they on said ever very and found said voice, she to had and Now, course with no low The its irons

and Therefore yourself And on let some Shed her It was argument but head see, out <s> for large all

least WAISTCOAT <s> Always you leaves, little Do with <s> Mabel was was upon baby argument is, LOVE and had HIM very by you I

earth house the processions you their never <s> looked and the she and she be Alice honour


# Bigram

In [16]:
bigram_lst=[]
for i in train_data:
    n = 2
    grams= ngrams(i.split(), n)
    for i in grams:
        bigram_lst.append(i)

In [17]:
bigram_ctr = Counter(bigram_lst)

In [18]:
def bigram_sentence_probability(sentence):
    try:
        p=1
        sentence = preprocessing(sentence)
        sentence = "<s> "+sentence+" </s>"
        keys =bigram_ctr.keys()
        p = p*unigram_mle[('<s>',)]
        prev_word = '<s>'
        for i in sentence.split()[1:]:

            if (prev_word,) in unigram_ctr and (prev_word,i) in bigram_ctr:
                p = p*bigram_ctr[(prev_word,i)]/unigram_ctr[(prev_word,)]
                prev_word = i
            else:
                return 0
        return math.log(p)
    
    except:
        return 0


def sample_word_bigram(prev_word):
    possible_word_probability = np.array([[i[1],bigram_ctr[i]*1.0/unigram_ctr[(prev_word,)]] for i in bigram_ctr.keys() if i[0]==prev_word])
    vals = possible_word_probability[:,1].astype(float)
    keys = possible_word_probability[:,0]
    sampled_word = keys[np.random.multinomial(1,vals)==1]
    return sampled_word[0]

def bigram_sentence_generator():
    prev_word = '<s>'
    sampled_word = sample_word_bigram(prev_word)
    prev_word = sampled_word
    s=""
    while sampled_word!='</s>':
        prev_word = sampled_word
        s=s+" "+sampled_word
        sampled_word = sample_word_bigram(prev_word)
       
    return s
    
    

In [19]:
bigram_sentence_probability('I want to')

-15.964102681360815

# Bigram MLE

In [20]:
bigram_mle={}
for i in bigram_ctr:
    bigram_mle[i] = bigram_ctr[i]*1.0/unigram_ctr[(i[0],)]
    

In [2]:
#bigram_mle

# Bigram Generator

In [22]:
for i in range(5):
    sent = bigram_sentence_generator()
    print ("\n"+sent)


 However, the Duchess

 The fourth

 Dinah was too late to find her ear, and she had been changed for I he cant help me

 Come, my throat

 Oh dear, and dishes


# Trigram

In [23]:
trigram_lst=[]
for i in train_data:
    n = 3
    grams= ngrams(i.split(), n)
    for i in grams:
        trigram_lst.append(i)

In [24]:
trigram_ctr = Counter(trigram_lst)

In [25]:
def trigram_sentence_probability(sentence):
    try:
        p=1
        sentence = preprocessing(sentence)
        sentence = "<s> "+sentence+" </s>"
        keys=trigram_ctr.keys()
        p = p*unigram_mle[('<s>',)]
        prev_word_1 = '<s>'
        p = p * bigram_ctr[(prev_word_1,sentence.split()[1])]/unigram_ctr[(prev_word_1,)]
        prev_word_2 = sentence.split()[1]
        for i in sentence.split()[2:]:
            #print ((prev_word_1,prev_word_2,i),p)
            if (prev_word_1,prev_word_2) in bigram_ctr and (prev_word_1,prev_word_2,i) in trigram_ctr:
                p = p*trigram_ctr[(prev_word_1,prev_word_2,i)]/bigram_ctr[(prev_word_1,prev_word_2)]

                prev_word_1 = prev_word_2
                prev_word_2 = i

            else:
                return 0

        return math.log(p)
    
    except:
        return 0



def trigram_sampler(prev_word_1,prev_word_2):
    possible_words = np.array([[i[2],trigram_ctr[i]*1.0/bigram_ctr[(i[0],i[1])]] for i in trigram_ctr.keys() if (i[0]==prev_word_1 and i[1]==prev_word_2)])
    keys = possible_words[:,0]
    vals = possible_words[:,1].astype(float)
    sampled_word = keys[np.random.multinomial(1,vals)==1]
    return sampled_word[0]
    
def trigram_sentence_generator():
    prev_word_1 = "<s>"
    prev_word_2 = sample_word_bigram(prev_word_1)
    sampled_word = trigram_sampler(prev_word_1,prev_word_2)
    s=""+prev_word_2
    while sampled_word!='</s>':
        s = s+" "+ sampled_word
        prev_word_1 = prev_word_2
        prev_word_2 = sampled_word
        sampled_word = trigram_sampler(prev_word_1,prev_word_2)
        
    return s
    
    
    



# Trigram MLE

In [26]:
trigram_mle={}
for i in trigram_ctr:
    trigram_mle[i] = trigram_ctr[i]*1.0/bigram_ctr[(i[0],i[1])]
    

In [27]:
s=0
for i in trigram_mle:
    if i[0]=='had' and i[1]=='any':
        s=s+trigram_mle[i]

In [3]:
#trigram_mle

# Trigram Generator

In [33]:
for i in range(5):
    sent = trigram_sentence_generator()
    print (sent+"\n")

Ugh, Serpent

said the Hatter grumbled you shouldnt talk, said the last time she heard it say to itself, Oh dear

Then they both sat silent and looked at them with the grin, which remained some time with one elbow against the door, staring stupidly up into a sort of lullaby to it

I dare say youre wondering why I dont want to be found all she could guess, she was appealed to by the Hatter, it woke up again

Who ARE you to death



# Quadragram


In [34]:
quadgram_lst=[]
for i in train_data:
    n = 4
    grams= ngrams(i.split(), n)
    for i in grams:
        quadgram_lst.append(i)

In [35]:
quadgram_ctr = Counter(quadgram_lst)

In [36]:
def quadgram_sentence_probability(sentence):
    try:
        prev_word_1 = '<s>'
        sentence = '<s> '+sentence+" </s>"
        p = 1
        p = p*unigram_mle[(prev_word_1,)]
        prev_word_2 = sentence.split()[1]
        prev_word_3 = sentence.split()[2]
        p = p * bigram_ctr[(prev_word_1,prev_word_2)]*1.0/unigram_ctr[(prev_word_1,)]
        p = p * trigram_ctr[(prev_word_1,prev_word_2,prev_word_3)]*1.0 /bigram_ctr[(prev_word_1,prev_word_2)]



        for i in sentence.split()[3:]:
            #print (prev_word_1,prev_word_2,prev_word_3,i,p,quadgram_ctr[(prev_word_1,prev_word_2,prev_word_3,i)])
            p = p*quadgram_ctr[(prev_word_1,prev_word_2,prev_word_3,i)]*1.0/trigram_ctr[(prev_word_1,prev_word_2,prev_word_3)]
            prev_word_1 = prev_word_2
            prev_word_2 = prev_word_3
            prev_word_3 = i
        return math.log(p)
    
    except:
        return 0

    
    
def quadgram_word_sampler(prev_word_1,prev_word_2,prev_word_3):
    possible_words = np.array([[i[3],quadgram_ctr[i]*1.0/trigram_ctr[(i[0],i[1],i[2])]] for i in quadgram_ctr.keys() if (i[0]==prev_word_1 and i[1]==prev_word_2 and i[2]==prev_word_3)])
    
    
    keys = possible_words[:,0]
    vals = possible_words[:,1].astype(float)
    sampled_word = keys[np.random.multinomial(1,vals)==1]
 
    return sampled_word[0]


def quadgram_word_generator():
    s=""
    prev_word_1 = "<s>"
    prev_word_2 = sample_word_bigram(prev_word_1)
    prev_word_3 = trigram_sampler(prev_word_1,prev_word_2)
    s= s+prev_word_2 + " "+prev_word_3+" "
    
    sampled_word = quadgram_word_sampler(prev_word_1,prev_word_2,prev_word_3)
    
    while sampled_word!="</s>":
        s= s+" "+sampled_word
        prev_word_1 = prev_word_2
        prev_word_2 = prev_word_3
        prev_word_3 = sampled_word
        sampled_word = quadgram_word_sampler(prev_word_1,prev_word_2,prev_word_3)
    return s
    
    

In [37]:
quadgram_sentence_probability("I wonder how many miles Ive fallen by this time")

-10.090464920866276

# Quadgram MLE

In [38]:
quadgram_mle={}
for i in quadgram_ctr:
    quadgram_mle[i] = quadgram_ctr[i]*1.0/trigram_ctr[(i[0],i[1],i[2])]
    

In [4]:
#quadgram_mle

# Quadgram Sentence Generator

In [59]:
for i in range(5):
    sent = quadgram_word_generator()
    print (sent+"\n")

You see  the earth takes twenty four hours to turn round on its axis Talking of axes, said the Duchess, and thats why

Dont go  splashing paint over me like that

While she  was looking at the house, and the March Hare

I wish  you were down here with me

This time  Alice waited patiently until it chose to speak again



# Model Comparision

In [41]:
unigram_sentence_probability("I wonder how many miles Ive fallen by this time",unigram_mle)

-74.84532521363464

In [42]:
bigram_sentence_probability("I wonder how many miles Ive fallen by this time")

-28.631919676231004

In [43]:
trigram_sentence_probability("I wonder how many miles Ive fallen by this time")

-13.453921974545585

In [44]:
quadgram_sentence_probability('I wonder how many miles Ive fallen by this time')

-10.090464920866276

Quadgram Model is superior compared to the other models

# Possible N-Grams


In [45]:
print ("The Number of Possible Unigrams are {} . Actual Number of Unigrams are {}".format(len(unigram_ctr),len(unigram_ctr)))

The Number of Possible Unigrams are 3201 . Actual Number of Unigrams are 3201


In [46]:
print ("The Number of Possible Bigrams are {} . Actual Number of Bigrams are {}".format(len(unigram_ctr)**2,len(bigram_ctr)))

The Number of Possible Bigrams are 10246401 . Actual Number of Bigrams are 13362


In [47]:
print ("The Number of Possible Trigrams are {} . Actual Number of Trigrams are {}".format(len(unigram_ctr)**3,len(trigram_ctr)))

The Number of Possible Trigrams are 32798729601 . Actual Number of Trigrams are 18779


In [48]:
print ("The Number of Possible Quadgrams are {} . Actual Number of Quadgrams are {}".format(len(unigram_ctr)**4,len(quadgram_ctr)))

The Number of Possible Quadgrams are 104988733452801 . Actual Number of Quadgrams are 19462


# Add one Smoothing


In [49]:
bigram_mle={}
for i in bigram_ctr:
    bigram_mle[i] = bigram_ctr[i]*1.0/unigram_ctr[(i[0],)]
    

In [5]:
#bigram_mle

In [51]:
import sys
bigram_add_one={}
for i in unigram_ctr:
    for j in unigram_ctr:
        if (i[0],j[0]) in bigram_ctr:
            bigram_add_one[(i[0],j[0])] = bigram_ctr[(i[0],j[0])]+1
        else:
            bigram_add_one[(i[0],j[0])] = 1
            
        
        

In [52]:
bigram_add_one_mle = {}
for i in bigram_add_one:
    bigram_add_one_mle[i] = bigram_add_one[i]*1.0/(unigram_ctr[(i[0],)]+len(unigram_ctr))
    

# Words that were affected drastically

In [53]:
random.seed(40)
word = random.choice(bigram_mle.keys())
print (word)
print ("Pre Smoothing Probability is ",bigram_mle[word])
print ("Post Smoothing Probability is ",bigram_add_one_mle[word])

('How', 'surprised')
('Pre Smoothing Probability is ', 0.041666666666666664)
('Post Smoothing Probability is ', 0.00062015503875969)


In [54]:
word = random.choice(bigram_mle.keys())
print (word)
print ("Pre Smoothing Probability is ",bigram_mle[word])
print ("Post Smoothing Probability is ",bigram_add_one_mle[word])

('then,', 'when')
('Pre Smoothing Probability is ', 0.08333333333333333)
('Post Smoothing Probability is ', 0.0006224712107065049)


In [55]:
random.seed(81)
word = random.choice(bigram_mle.keys())
print (word)
print ("Pre Smoothing Probability is ",bigram_mle[word])
print ("Post Smoothing Probability is ",bigram_add_one_mle[word])

('Rabbits', 'voice')
('Pre Smoothing Probability is ', 0.5)
('Post Smoothing Probability is ', 0.00093603744149766)


# Good Turing Smoothing

In [64]:
original_ctr = Counter(bigram_ctr.values())
gt_ctr={}

In [83]:
for i in range(1,11):
    gt_ctr[i] = (i+1)*(original_ctr[i+1])*1.0/(original_ctr[i])
    

In [88]:
for i in gt_ctr:
    print (i - gt_ctr[i])

0.718266844818
0.908536585366
1.05586592179
1.06896551724
0.960784313725
1.5145631068
1.06060606061
2.30612244898
-1.96774193548
4.17647058824


The Value of Discounting Factor is close to 1. 

# Perplexity Analysis

In [75]:
perplexity()

In [79]:
for j in range(2,11):
    s=s+gt_ctr[j]*(j-1)

In [80]:
s

4731.250137759335